TGBOT & GIGACHAT

Requirements

In [ ]:
!pip install -q langchain_community
!pip install telebot
!pip install GigaChat

Необходимые импорты

In [2]:
from langchain_community.chat_models.gigachat import GigaChat
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import types

In [3]:
# Словарь для хранения ConversationBufferMemory каждого пользователя
user_conversations = {}

In [4]:
import telebot
from time import sleep

bot_token = "7812195126:AAENUXACs8OER1JGiieR45b9ekRWqK8UOsI"
bot = telebot.TeleBot(bot_token)

SBER_AUTH = "YWE0ODFhNDctOTI3NS00YmI4LWJjMjctZDQ2NDk3ODgwN2E5OmE1NjNjMjNiLTdiMzItNDQ5Yy1iOTJiLTc2YTcyMzdlZGU3OQ=="
sber = SBER_AUTH

In [ ]:
# Инициализация GigaChat и ConversationChain
llm = GigaChat(credentials=sber, verify_ssl_certs=False)
conversation = ConversationChain(llm=llm,
                                 verbose=True,
                                 memory=ConversationBufferMemory())

In [8]:
from telebot import types

# Словарь для хранения состояния пользователя
user_states = {}

# Функция для автоматического ответа в случае нетекстового сообщения
@bot.message_handler(content_types=['audio', 'video', 'document', 'photo', 'sticker', 'voice', 'location', 'contact'])
def not_text(message):
    user_id = message.chat.id
    bot.send_message(user_id, 'Я работаю только с текстовыми сообщениями!')

# Начальный обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    user_id = message.chat.id
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add('Использовать свой текст', 'Сгенерировать текст')
    bot.send_message(user_id, "Выберите действие:", reply_markup=markup)
    user_states[user_id] = 'choose_action'

# Обработчик выбора действия
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'choose_action')
def handle_choice(message):
    user_id = message.chat.id
    if message.text == 'Использовать свой текст':
        bot.send_message(user_id, "Введите свой текст:")
        user_states[user_id] = 'user_text_input'
    elif message.text == 'Сгенерировать текст':
        bot.send_message(user_id, "Что бы вы хотели услышать? (начните свой запрос со слов \"сгенерируй...\")")
        user_states[user_id] = 'awaiting_generation_request'
    else:
        bot.send_message(user_id, "Пожалуйста, выберите действие из меню.")

# Обработчик для пользовательского текста
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'user_text_input')
def handle_user_text_input(message):
    user_id = message.chat.id
    # Просто отображаем текст без обращения к GigaChat
    bot.send_message(user_id, f"Ваш текст принят!")
    # Сбрасываем состояние после обработки
    user_states.pop(user_id, None)

# Обработчик для генерации текста через GigaChat
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'awaiting_generation_request')
def handle_generation_request(message):
    user_id = message.chat.id
    if message.text.lower().startswith("сгенерируй"):
        # Отправка запроса в GigaChat
        response = conversation.predict(input=message.text)
        bot.send_message(user_id, response)
        # Выводим дополнительное сообщение
        bot.send_message(user_id, "Отлично! Теперь у вас есть текст, скопируйте его и начните сначала через команду /start, затем выберите \"Использовать свой текст\"")
        # Сбрасываем состояние после обработки
        user_states.pop(user_id, None)
    else:
        bot.send_message(user_id, 'Пожалуйста, начните свой запрос со слов "сгенерируй...".')

# Если состояние не распознано, просим пользователя начать заново
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) is None)
def ask_to_start(message):
    bot.send_message(message.chat.id, "Пожалуйста, используйте команду /start для начала.")


In [ ]:
# Запуск бота
bot.polling(none_stop=True)